In [5]:
import json
import requests
import numpy as np
import pandas as pd

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidentes = json.loads(response.text)
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
dist_est_barrios = json.loads(response.text)
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
dist_est = json.loads(response.text)
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
barrios = json.loads(response.text)
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
estaciones = json.loads(response.text)

In [6]:
dic_density = {} # diccionario de densidades de barrios precalculado

In [7]:
for b in barrios:
    dic_density[b['nta']] = round( (b['population'] / b['shape_area'])*100 , 5)

In [8]:
dic_incid = {} # diccionario de incidentes precalculado.

In [9]:
for i in incidentes:
    barri = i['nta'] # barrio del incidente
    if barri not in dic_incid: # si el barrio no se guardó aún
        dic_incid[barri] = {} # dic dentro de cada barrio
        # incidentes de noche y de día con dic de conteo de incidentes
        dic_incid[barri]['noche'] = {'num_incid': 0}
        dic_incid[barri]['dia'] = {'num_incid': 0}
    turno = 'noche'
    if i['is_first_shift'] == False: # si el incidente es de día
        turno = 'dia'

    dic_incid[barri][turno]['num_incid'] += 1 # aumentamos contador

    incid_pos = dic_incid[barri][turno]['num_incid']
    # cogemos valor de incidente y lo usamos para crear lista de tipos de vehículos
    # cada posición de la lista es un tipo de vehículo.
    dic_incid[barri][turno][f'incidente_{incid_pos}'] = [0,0,0,0,0]

    units = i['units'] # guardamos tipos de vehiculo del incidente
    for u in set(units): # set para sacar valores únicos
        # guardamos valores de vaca tipo de vehículo por incidente
        if u == 'engine':
          dic_incid[barri][turno][f'incidente_{incid_pos}'][0] += units.count(u)
        elif u == 'ladder':
          dic_incid[barri][turno][f'incidente_{incid_pos}'][1] += units.count(u)
        elif u == 'rescue':
          dic_incid[barri][turno][f'incidente_{incid_pos}'][2] += units.count(u)
        elif u == 'squad':
          dic_incid[barri][turno][f'incidente_{incid_pos}'][3] += units.count(u)
        elif u == 'hazardous':
          dic_incid[barri][turno][f'incidente_{incid_pos}'][4] += units.count(u)

In [10]:
dic_ests  = {} # diccionario para guardar las estaciones cercanas a cada

In [12]:
dic_ests['101-02 Jamaica Avenue']

{'barrios_cercanos': {'QN08': [726.54,
   {'noche': {'num_incid': 1236,
     'incidente_1': [0, 1, 0, 0, 0],
     'incidente_2': [0, 1, 0, 0, 0],
     'incidente_3': [1, 0, 0, 0, 0],
     'incidente_4': [0, 0, 1, 0, 0],
     'incidente_5': [1, 0, 0, 0, 0],
     'incidente_6': [1, 0, 0, 0, 0],
     'incidente_7': [1, 0, 0, 0, 0],
     'incidente_8': [1, 1, 0, 0, 0],
     'incidente_9': [1, 0, 0, 0, 0],
     'incidente_10': [2, 3, 0, 0, 0],
     'incidente_11': [0, 1, 0, 0, 0],
     'incidente_12': [0, 1, 0, 0, 0],
     'incidente_13': [1, 0, 0, 0, 0],
     'incidente_14': [1, 0, 0, 0, 0],
     'incidente_15': [0, 1, 0, 0, 0],
     'incidente_16': [0, 3, 0, 0, 0],
     'incidente_17': [2, 0, 0, 0, 0],
     'incidente_18': [2, 1, 0, 0, 0],
     'incidente_19': [0, 0, 0, 0, 1],
     'incidente_20': [0, 1, 0, 0, 0],
     'incidente_21': [1, 2, 0, 0, 0],
     'incidente_22': [1, 0, 0, 0, 0],
     'incidente_23': [1, 0, 0, 0, 0],
     'incidente_24': [1, 0, 0, 0, 0],
     'incidente_25': [2, 

In [11]:
barrios_cerca, densidades, incids_noche, incids_dia = [],[],[],[]
vhs_noche, vhs_dia, capacidades, estaciones_cerca = [],[],[],[]
nom_estaciones = []

for i in estaciones:
    ### NOMBRE DE LA ESTACIÓN
    dir = i['address']
    dic_ests[dir] = {}

    nom_estaciones.append(dir)

    ### BARRIOS CERCANOS (<= 780 segs)
    dic_ests[dir]['barrios_cercanos'] = {}
    for b,d in dist_est_barrios[dir].items():
        if d <= 780: # menos de esta dist no en algunas no entran barrios.
            dic_ests[dir]['barrios_cercanos'][b] = [d]

    barrios_cerca.append(len(dic_ests[dir]['barrios_cercanos']))

    ### BARRIO DE LA ESTACIÓN CON NOMBRE, DISTANCIA Y DENSIDAD
    dist, barrio = min((v[0],k) for k,v in dic_ests[dir]['barrios_cercanos'].items())
    dic_ests[dir]['barrio'] = [barrio, dist, dic_density[barrio]]

    densidades.append(dic_density[barrio])

    ### INCIDENTES EN BARRIOS CERCANOS
    for b,d in dic_ests[dir]['barrios_cercanos'].items():
        if b in dic_incid.keys():
          d.append(dic_incid[b])
        else:
          d.append({})

    ### INCIDENTES MÁXIMOS DE DÍA Y DE NOCHE EN UN BARRIO
    incid_mean_noche, incid_mean_dia = [],[]
    for b,d in dic_ests[dir]['barrios_cercanos'].items():
      if d[1] != {}:
          if d[1]['noche'] != {}:
              incid_mean_noche.append((d[1]['noche']['num_incid'], b))
          else:
              incid_mean_noche.append((0, b))
          if d[1]['dia'] != {}:
              incid_mean_dia.append((d[1]['dia']['num_incid'], b))
          else:
              incid_mean_dia.append((0, b))
      else:
          incid_mean_noche.append((0, b))
          incid_mean_dia.append((0, b))

    incid_mean_noche = int(np.mean([x[0] for x in incid_mean_noche]))
    incid_mean_dia = int(np.mean([x[0] for x in incid_mean_dia]))

    dic_ests[dir]['incidentes_max'] = {'noche': incid_mean_noche,
                                       'dia': incid_mean_dia }

    incids_noche.append(incid_mean_noche)
    incids_dia.append(incid_mean_dia)

    ### TIPOS DE VEHÍCULOS EN BASE A INCIDENTES
    for b,d in dic_ests[dir]['barrios_cercanos'].items():
      if d[1] != {}:
          mean_vh_noche = np.mean(np.array([ v if (k != 'num_incid' and d[1]['noche'] is not None) else [0,0,0,0,0]
                                                      for k,v in d[1]['noche'].items() ]), axis=0)
          mean_vh_dia = np.mean(np.array([ v if (k != 'num_incid' and d[1]['dia'] is not None) else [0,0,0,0,0]
                                                      for k,v in d[1]['dia'].items() ]), axis=0)
      else:
          continue
      dic_ests[dir]['media_tipo_vhs'] = {
              'noche': list(mean_vh_noche),
              'dia': list(mean_vh_dia)
          }

    vhs_noche.append(list(mean_vh_noche))
    vhs_dia.append(list(mean_vh_dia))

    ### CAPACIDAD DE LA ESTACIÓN
    dic_ests[dir]['capacidad'] = i['capacity']

    capacidades.append(dic_ests[dir]['capacidad'])

    ### ESTACIONES CERCANAS A ESA ESTACIÓN (<= 600 seg)
    dic_ests[dir]['estaciones_cercanas'] = []
    for e,d in dist_est[dir].items():
        if d <= 600:
            dic_ests[dir]['estaciones_cercanas'].append((e,d))

    estaciones_cerca.append(len(dic_ests[dir]['estaciones_cercanas']))

In [34]:
for i,v in dic_ests.items():
  if len(v["estaciones_cercanas"]) < 4:
    print(len(v["estaciones_cercanas"]))

0
3
3
3
2
2
3
3
3
3


### Cálculo de los Scorings

In [8]:
list_scores_1 = [ barrios_cerca, densidades, incids_noche, incids_dia,
                capacidades, estaciones_cerca ] # VALORES ÚNICOS
list_scores_2 = [ vhs_noche, vhs_dia ] # LISTA DE VALORES

In [9]:
### NORMALIZAR SCORES PARA VALORES ÚNICOS
def normalize_list(numbers):
    min_val = min(numbers)
    max_val = max(numbers)
    normalized = [(x - min_val) / (max_val - min_val) for x in numbers]
    return normalized

In [10]:
### NORMALIZAR SCORES PARA VALORES EN LISTAS
def normalize_list_of_lists(list_of_lists):
    normalized_list_of_lists = []
    for sublist in list_of_lists:
        min_val = min(sublist)
        max_val = max(sublist)
        normalized_sublist = [(x - min_val) / (max_val - min_val)
                                    if sublist != [0,0,0,0,0] else 0 for x in sublist]
        normalized_list_of_lists.append(normalized_sublist)
    return normalized_list_of_lists

In [11]:
list_scores_norm = []

In [12]:
for l in list_scores_1:
    list_scores_norm.append(normalize_list(l))
for l in list_scores_2:
    list_scores_norm.append(normalize_list_of_lists(l))

In [13]:
### AÑADIR SCORES A CADA ESTACIÓN
for e in range(len(nom_estaciones)):
    dic_ests[nom_estaciones[e]]['scorings'] = {
        'num_barrios_cerca': list_scores_norm[0][e],
        'densidad': list_scores_norm[1][e],
        'mean_incids_noche': list_scores_norm[2][e],
        'mean_incids_dia': list_scores_norm[3][e],
        'capacidad': list_scores_norm[4][e],
        'num_estaciones_cerca': list_scores_norm[5][e],
        'mean_vhs_noche_tipo': {
            'tipo_1': list_scores_norm[6][e][0],
            'tipo_2': list_scores_norm[6][e][1],
            'tipo_3': list_scores_norm[6][e][2],
            'tipo_4': list_scores_norm[6][e][3],
            'tipo_5': list_scores_norm[6][e][4]
        },
        'mean_vhs_dia_tipo': {
            'tipo_1': list_scores_norm[7][e][0],
            'tipo_2': list_scores_norm[7][e][1],
            'tipo_3': list_scores_norm[7][e][2],
            'tipo_4': list_scores_norm[7][e][3],
            'tipo_5': list_scores_norm[7][e][4]
        }
    }

In [14]:
dic_ests['100 Duane Street']['scorings']

{'num_barrios_cerca': 0.4418604651162791,
 'densidad': 0.3808790030818706,
 'mean_incids_noche': 0.32701421800947866,
 'mean_incids_dia': 0.595360824742268,
 'capacidad': 0.75,
 'num_estaciones_cerca': 0.6176470588235294,
 'mean_vhs_noche_tipo': {'tipo_1': 0.8317329675354367,
  'tipo_2': 1.0,
  'tipo_3': 0.050297210791037945,
  'tipo_4': 0.00045724737082761735,
  'tipo_5': 0.0},
 'mean_vhs_dia_tipo': {'tipo_1': 1.0,
  'tipo_2': 0.9894810659186536,
  'tipo_3': 0.012622720897615708,
  'tipo_4': 0.0,
  'tipo_5': 0.06311360448807854}}

In [15]:
# Definir las importancias de cada scoring
importancias = {
    'num_barrios_cerca': 0.25,
    'densidad': 0.05,
    'mean_incids_noche': 0.25,
    'mean_incids_dia': 0.25,
    'capacidad': 0.15,
    'num_estaciones_cerca': 0.2,
    'mean_vhs_noche_tipo': 0.1,
    'mean_vhs_dia_tipo': 0.1
}

def calcular_scoring_general(estaciones):
    resultados = {}

    for nombre, datos in dic_ests.items():
        scorings = datos['scorings']

        # Calcular el scoring para la noche
        scoring_noche = (
            scorings['num_barrios_cerca'] * importancias['num_barrios_cerca'] +
            scorings['densidad'] * importancias['densidad'] +
            scorings['mean_incids_noche'] * importancias['mean_incids_noche'] +
            scorings['capacidad'] * importancias['capacidad'] +
            scorings['num_estaciones_cerca'] * importancias['num_estaciones_cerca'] +
            sum(scorings['mean_vhs_noche_tipo'].values()) * importancias['mean_vhs_noche_tipo']
        )

        # Calcular el scoring para el día
        scoring_dia = (
            scorings['num_barrios_cerca'] * importancias['num_barrios_cerca'] +
            scorings['densidad'] * importancias['densidad'] +
            scorings['mean_incids_dia'] * importancias['mean_incids_dia'] +
            scorings['capacidad'] * importancias['capacidad'] +
            scorings['num_estaciones_cerca'] * importancias['num_estaciones_cerca'] +
            sum(scorings['mean_vhs_dia_tipo'].values()) * importancias['mean_vhs_dia_tipo']
        )

        resultados[nombre] = {'scoring_noche': scoring_noche, 'scoring_dia': scoring_dia}

    # Paso 2: Encontrar los valores mínimos y máximos
    min_scoring = min(resultados[nombre]['scoring_noche'] for nombre in resultados)
    max_scoring = max(resultados[nombre]['scoring_noche'] for nombre in resultados)
    min_scoring_dia = min(resultados[nombre]['scoring_dia'] for nombre in resultados)
    max_scoring_dia = max(resultados[nombre]['scoring_dia'] for nombre in resultados)

    # Paso 3: Aplicar normalización min-max
    for nombre in resultados:
        scoring_noche = resultados[nombre]['scoring_noche']
        scoring_dia = resultados[nombre]['scoring_dia']
        resultados[nombre]['scoring_noche'] = (scoring_noche - min_scoring) / (max_scoring - min_scoring) if max_scoring != min_scoring else 1
        resultados[nombre]['scoring_dia'] = (scoring_dia - min_scoring_dia) / (max_scoring_dia - min_scoring_dia) if max_scoring_dia != min_scoring_dia else 1

    return resultados

In [16]:
scorings_generales = calcular_scoring_general(dic_ests)

In [17]:
print('Mínimo:', min(v2 for k,v in scorings_generales.items() for t,v2 in v.items()))
print('Máximo:', max(v2 for k,v in scorings_generales.items() for t,v2 in v.items()))

Mínimo: 0.0
Máximo: 1.0


In [18]:
for k,v in scorings_generales.items():
  print(k, v)

42 South Street {'scoring_noche': 0.5337281970110397, 'scoring_dia': 0.7003853285875802}
49 Beekman Street {'scoring_noche': 0.499099033285966, 'scoring_dia': 0.6769983705197898}
100 Duane Street {'scoring_noche': 0.726813622426507, 'scoring_dia': 0.8979562770213142}
14 N. Moore Street {'scoring_noche': 0.522140645165827, 'scoring_dia': 0.7565565688362432}
75 Canal Street {'scoring_noche': 0.7197296843555254, 'scoring_dia': 0.8299004434494419}
25 Pitt Street {'scoring_noche': 0.7487851564461535, 'scoring_dia': 0.8516944786033648}
222 East 2 Street {'scoring_noche': 0.6738621036933432, 'scoring_dia': 0.8598306834938367}
340 East 14 Street {'scoring_noche': 0.6566567938493638, 'scoring_dia': 0.877699435719743}
253 Lafayette Street {'scoring_noche': 0.6116774098994008, 'scoring_dia': 0.834822071618958}
42 Great Jones Street {'scoring_noche': 0.6878695922984656, 'scoring_dia': 0.9107799833933585}
108 East 13 Street {'scoring_noche': 0.6917551544303925, 'scoring_dia': 0.9135565897714117}
13